In [1]:
import time
from typing import Any
from omegaconf import OmegaConf
from codebase.stool import StoolArgs, launch_job

In [2]:
date = time.strftime("%m%d-%H%M%S")
capella_args = StoolArgs(
    config="./experiments/baseline_transformer/configs/debug.yaml",
    script="experiments.baseline_transformer.train",
    copy_code= False,
    nodes = 1,  # The number of nodes to run the job on.
    ngpu = 1,  # The number of GPUs required per node.
    ncpu = 32,  # The number of CPUs allocated per GPU.
    mem = "32Gb",
    anaconda = "/data/horse/ws/lama722b-nanite-lm/nanite-lm/.env",
    time = 60,
    account = "p_scads",
    partition = "capella"
)
capella_args.config = OmegaConf.load(capella_args.config)
capella_args.config['dump_dir'] = f"/data/horse/ws/lama722b-nanite-lm/dumps/{date}"

In [3]:
launch_job(capella_args)

Creating directories...
Saving config file ...
Writing sbatch command ...
> /data/horse/ws/lama722b-nanite-lm/nanite-lm/codebase/stool.py(211)launch_job()
    209     import pdb; pdb.set_trace()
    210 
--> 211     print("Submitting job ...")
    212     os.system(f"{args.launcher} {dump_dir}/submit.slurm")
    213 



ipdb>  exit
